In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier as GBDTc
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import cross_validation,metrics
from sklearn import svm
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

offline_train_df = pd.read_csv('ccf_offline_stage1_train/ccf_offline_stage1_train.csv')
test_df = pd.read_csv('ccf_offline_stage1_test_revised.csv')

c:\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
offline_train_df.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN


In [3]:
test_df.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1.0,20160712
1,6949378,1300,3429,30:5,NaN,20160706
2,2166529,7113,6928,200:20,5.0,20160727
3,2166529,7113,1808,100:10,5.0,20160727
4,6172162,7605,6500,30:1,2.0,20160708


In [4]:
test_df.count()

User_id          113640
Merchant_id      113640
Coupon_id        113640
Discount_rate    113640
Distance         101576
Date_received    113640
dtype: int64

In [5]:
offline_train_df.count()

User_id          1754884
Merchant_id      1754884
Coupon_id        1053282
Discount_rate    1053282
Distance         1648881
Date_received    1053282
Date              776984
dtype: int64

In [6]:
offline_train_df[['Merchant_id', 'User_id']].groupby(['Merchant_id'], as_index=True).size()

Merchant_id
1        14
2        11
3        18
4        40
5        79
6        61
7         3
8         2
9         6
10        1
11       11
12        6
13       49
14       26
15      194
17       96
18       29
19       96
20      204
21       23
22       15
23       25
24        6
25      286
26       15
27       18
28       49
29        8
30        8
31       50
       ... 
8827     14
8828     63
8829      4
8830      8
8831    355
8832      1
8833      9
8834      7
8835     57
8836      8
8837     38
8838      8
8839     23
8840    104
8841     13
8842     19
8843     79
8844    822
8845     26
8846     40
8847     30
8848     42
8849      6
8850     33
8851      4
8852     28
8853     40
8854     21
8855     14
8856    250
Length: 8415, dtype: int64

In [7]:
offline_train_df.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN


In [8]:
offline_train_df.loc[offline_train_df['Distance'].isnull()]['Date'][offline_train_df.loc[offline_train_df['Distance'].isnull()]['Date'].isnull()]

13        NaN
17        NaN
24        NaN
52        NaN
58        NaN
112       NaN
130       NaN
131       NaN
132       NaN
133       NaN
134       NaN
135       NaN
136       NaN
139       NaN
140       NaN
154       NaN
209       NaN
217       NaN
280       NaN
380       NaN
381       NaN
403       NaN
404       NaN
442       NaN
508       NaN
513       NaN
523       NaN
577       NaN
598       NaN
602       NaN
           ..
1754497   NaN
1754531   NaN
1754541   NaN
1754542   NaN
1754574   NaN
1754638   NaN
1754685   NaN
1754686   NaN
1754687   NaN
1754688   NaN
1754689   NaN
1754690   NaN
1754697   NaN
1754702   NaN
1754706   NaN
1754720   NaN
1754729   NaN
1754730   NaN
1754741   NaN
1754753   NaN
1754760   NaN
1754761   NaN
1754768   NaN
1754769   NaN
1754770   NaN
1754771   NaN
1754772   NaN
1754844   NaN
1754848   NaN
1754849   NaN
Name: Date, Length: 97786, dtype: float64

In [9]:
offline_train_df.loc[offline_train_df['Distance'].isnull()]['Date'].isnull().count()

106003

In [10]:
(106003 - 97786)/106003

0.07751667405639463

In [11]:
offline_train_df['Distance'] = offline_train_df['Distance'].fillna(11)

In [12]:
offline_train_df['Date'] = offline_train_df['Date'].fillna(0)

In [13]:
offline_train_df['Date_received'] = offline_train_df['Date_received'].fillna(0)

In [14]:
offline_train_df.isnull().any()

User_id          False
Merchant_id      False
Coupon_id         True
Discount_rate     True
Distance         False
Date_received    False
Date             False
dtype: bool

In [15]:
def label(row):
    if row['Date_received'] == 0 :
        return -1
    if not row['Date'] == 0:
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0
offline_train_df['within_15'] = offline_train_df.apply(label, axis=1)

In [16]:
print(offline_train_df['within_15'].value_counts())

 0    988887
-1    701602
 1     64395
Name: within_15, dtype: int64


In [17]:
test_df['Distance'] = test_df['Distance'].fillna(11)

In [18]:
offline_train_df['Distance'] = offline_train_df['Distance'].astype(float)
test_df['Distance'] = test_df['Distance'].astype(float)

In [19]:
test_df.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1.0,20160712
1,6949378,1300,3429,30:5,11.0,20160706
2,2166529,7113,6928,200:20,5.0,20160727
3,2166529,7113,1808,100:10,5.0,20160727
4,6172162,7605,6500,30:1,2.0,20160708


In [20]:
offline_train_df['Discount_rate'].unique()

array([nan, '150:20', '20:1', '200:20', '30:5', '50:10', '10:5', '100:10',
       '200:30', '20:5', '30:10', '50:5', '150:10', '100:30', '200:50',
       '100:50', '300:30', '50:20', '0.9', '10:1', '30:1', '0.95',
       '100:5', '5:1', '100:20', '0.8', '50:1', '200:10', '300:20',
       '100:1', '150:30', '300:50', '20:10', '0.85', '0.6', '150:50',
       '0.75', '0.5', '200:5', '0.7', '30:20', '300:10', '0.2', '50:30',
       '200:100', '150:5'], dtype=object)

In [21]:
offline_train_df.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,within_15
0,1439408,2632,NaN,NaN,0.0,0.0,20160217.0,-1
1,1439408,4663,11002.0,150:20,1.0,20160528.0,0.0,0
2,1439408,2632,8591.0,20:1,0.0,20160217.0,0.0,0
3,1439408,2632,1078.0,20:1,0.0,20160319.0,0.0,0
4,1439408,2632,8591.0,20:1,0.0,20160613.0,0.0,0


In [22]:
test_df.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1.0,20160712
1,6949378,1300,3429,30:5,11.0,20160706
2,2166529,7113,6928,200:20,5.0,20160727
3,2166529,7113,1808,100:10,5.0,20160727
4,6172162,7605,6500,30:1,2.0,20160708


In [23]:
proba = offline_train_df[['Merchant_id', 'within_15']].groupby('Merchant_id').mean().sort_values('within_15', ascending=False)

In [24]:
offline_train_df['proba'] = [1 if proba.loc[x]['within_15']> 0 else 0 for x in offline_train_df['Merchant_id']]

In [25]:
def discount_type(row):
    if ':' in str(row['Discount_rate']):
        return 1
    else:
        return 0
offline_train_df['Dicount_type'] = offline_train_df.apply(discount_type, axis=1)

In [26]:
def discount_shop(row):
    if ':' in str(row['Discount_rate']):
        return int(row['Discount_rate'].split(':')[0])
    else:
        return -1
offline_train_df['Dicount_shop'] = offline_train_df.apply(discount_shop, axis=1)

In [27]:
def discount_number(row):
    if ':' in str(row['Discount_rate']):
        return int(row['Discount_rate'].split(':')[1])
    else:
        return -1
offline_train_df['Dicount_number'] = offline_train_df.apply(discount_number, axis=1)

In [28]:
def discount_type(row):
    if ':' in str(row['Discount_rate']):
        return 1
    else:
        return 0
test_df['Dicount_type'] = test_df.apply(discount_type, axis=1)

In [29]:
def discount_shop(row):
    if ':' in str(row['Discount_rate']):
        return int(row['Discount_rate'].split(':')[0])
    else:
        return -1
test_df['Dicount_shop'] = test_df.apply(discount_shop, axis=1)

In [30]:
def discount_number(row):
    if ':' in str(row['Discount_rate']):
        return int(row['Discount_rate'].split(':')[1])
    else:
        return -1
test_df['Dicount_number'] = test_df.apply(discount_number, axis=1)

In [31]:
offline_train_df.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,within_15,proba,Dicount_type,Dicount_shop,Dicount_number
0,1439408,2632,NaN,NaN,0.0,0.0,20160217.0,-1,0,0,-1,-1
1,1439408,4663,11002.0,150:20,1.0,20160528.0,0.0,0,0,1,150,20


In [32]:
title_mapping = {}
for i in offline_train_df['Discount_rate']:
    if ':' in str(i):
        title_mapping[i] = (int(i.split(':')[0]) - int(i.split(':')[1])) /  int(i.split(':')[0])
    else:
        title_mapping[i] = i
offline_train_df['Discount_rate'] = offline_train_df['Discount_rate'].map(title_mapping)
test_df['Discount_rate'] = test_df['Discount_rate'].map(title_mapping)

In [33]:
test_df.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Dicount_type,Dicount_shop,Dicount_number
0,4129537,450,9983,0.833333,1.0,20160712,1,30,5
1,6949378,1300,3429,0.833333,11.0,20160706,1,30,5


In [34]:
from datetime import date
def getWeekday(row):
    if row == '0.0':
        return row
    else:
        return date(int(row[0:4]), int(row[4:6]), int(row[6:8])).weekday() + 1

offline_train_df['weekday'] = offline_train_df['Date_received'].astype(str).apply(getWeekday)
test_df['weekday'] = test_df['Date_received'].astype(str).apply(getWeekday)

# weekday_type :  周六和周日为1，其他为0
offline_train_df['weekday_type'] = offline_train_df['weekday'].apply(lambda x : 1 if x in [6,7] else 0 )
test_df['weekday_type'] = test_df['weekday'].apply(lambda x : 1 if x in [6,7] else 0 )

In [35]:
offline_train_df['weekday'].unique()

array(['0.0', 6, 3, 1, 5, 4, 7, 2], dtype=object)

In [36]:
# change weekday to one-hot encoding 
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
print(weekdaycols)

tmpdf = pd.get_dummies(offline_train_df['weekday'].replace('0.0', np.nan))
print(tmpdf)
tmpdf.columns = weekdaycols
offline_train_df[weekdaycols] = tmpdf

tmpdf = pd.get_dummies(test_df['weekday'].replace('0.0', np.nan))
tmpdf.columns = weekdaycols
test_df[weekdaycols] = tmpdf

['weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']
         1.0  2.0  3.0  4.0  5.0  6.0  7.0
0          0    0    0    0    0    0    0
1          0    0    0    0    0    1    0
2          0    0    1    0    0    0    0
3          0    0    0    0    0    1    0
4          1    0    0    0    0    0    0
5          0    0    0    0    0    0    0
6          1    0    0    0    0    0    0
7          0    0    0    0    1    0    0
8          0    0    0    0    1    0    0
9          1    0    0    0    0    0    0
10         0    0    0    1    0    0    0
11         0    0    0    0    0    0    0
12         0    0    0    0    0    0    0
13         1    0    0    0    0    0    0
14         0    0    0    0    0    0    0
15         1    0    0    0    0    0    0
16         0    0    0    0    1    0    0
17         0    0    0    0    0    0    1
18         0    0    0    1    0    0    0
19         0    0    0    0    0    1    0
20   

In [37]:
# offline_train_df['Discount_rate'] = offline_train_df['Discount_rate'].astype(float)
# offline_train_df['Discount_rate'] = offline_train_df['Discount_rate'] / offline_train_df['Discount_rate'].max()
# test_df['Discount_rate'] = test_df['Discount_rate'].astype(float)
# test_df['Discount_rate'] = test_df['Discount_rate'] / test_df['Discount_rate'].max()

In [38]:
test_df.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Dicount_type,Dicount_shop,Dicount_number,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
0,4129537,450,9983,0.833333,1.0,20160712,1,30,5,2,0,0,1,0,0,0,0,0
1,6949378,1300,3429,0.833333,11.0,20160706,1,30,5,3,0,0,0,1,0,0,0,0


In [39]:
off_Merchant = [x[1] for x in offline_train_df.values if x[8]>=1]
test_df['proba'] = [1 if x in off_Merchant else 0 for x in test_df['Merchant_id']]

In [40]:
offline_train_df.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,within_15,proba,Dicount_type,...,Dicount_number,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
0,1439408,2632,NaN,NaN,0.0,0.0,20160217.0,-1,0,0,...,-1,0.0,0,0,0,0,0,0,0,0
1,1439408,4663,11002.0,0.866667,1.0,20160528.0,0.0,0,0,1,...,20,6,1,0,0,0,0,0,1,0
2,1439408,2632,8591.0,0.95,0.0,20160217.0,0.0,0,0,1,...,1,3,0,0,0,1,0,0,0,0
3,1439408,2632,1078.0,0.95,0.0,20160319.0,0.0,0,0,1,...,1,6,1,0,0,0,0,0,1,0
4,1439408,2632,8591.0,0.95,0.0,20160613.0,0.0,0,0,1,...,1,1,0,1,0,0,0,0,0,0


In [41]:
offline_train_df = offline_train_df.drop('Date', axis =1)

In [42]:
# offline_train_df['User_id'] = offline_train_df['User_id'].astype(float)
# offline_train_df['User_id'] = offline_train_df['User_id'] / offline_train_df['User_id'].max()
# offline_train_df['Date_received'] = offline_train_df['Date_received'].astype(float)
# offline_train_df['Date_received'] = offline_train_df['Date_received'] / offline_train_df['Date_received'].max()
# offline_train_df['Merchant_id'] = offline_train_df['Merchant_id'].astype(float)
# offline_train_df['Merchant_id'] = offline_train_df['Merchant_id'] / offline_train_df['Merchant_id'].max()

In [43]:
offline_train_df['Discount_rate'] = offline_train_df['Discount_rate'].fillna(-1)

In [44]:
offline_train_df.isnull().any()

User_id           False
Merchant_id       False
Coupon_id          True
Discount_rate     False
Distance          False
Date_received     False
within_15         False
proba             False
Dicount_type      False
Dicount_shop      False
Dicount_number    False
weekday           False
weekday_type      False
weekday_1         False
weekday_2         False
weekday_3         False
weekday_4         False
weekday_5         False
weekday_6         False
weekday_7         False
dtype: bool

In [45]:
offline_train_df.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,within_15,proba,Dicount_type,Dicount_shop,Dicount_number,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
0,1439408,2632,NaN,-1,0.0,0.0,-1,0,0,-1,-1,0.0,0,0,0,0,0,0,0,0
1,1439408,4663,11002.0,0.866667,1.0,20160528.0,0,0,1,150,20,6,1,0,0,0,0,0,1,0


In [46]:
offline_train_df['Coupon_id'] = offline_train_df['Coupon_id'].fillna(-1)

In [47]:
test_df.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Dicount_type,Dicount_shop,Dicount_number,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,proba
0,4129537,450,9983,0.833333,1.0,20160712,1,30,5,2,0,0,1,0,0,0,0,0,0
1,6949378,1300,3429,0.833333,11.0,20160706,1,30,5,3,0,0,0,1,0,0,0,0,0


In [69]:
train_x,test_x,train_y,test_y =cross_validation.train_test_split(offline_train_df.drop(['within_15','User_id', 'Merchant_id', 'Date_received'], axis=1),offline_train_df['within_15'],test_size=0.2,random_state=27)#把训练集按0.2的比例划分为训练集和验证集
#start svmGaussianNB
clf = GBDTc(n_estimators=10)

clf.fit(train_x.drop('Coupon_id', axis=1), train_y)

prodict_prob_y = clf.predict_proba(test_x.drop('Coupon_id', axis=1))

In [70]:
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve

test_x['pred_prob'] = prodict_prob_y.T[0]
test_x['within_15'] = test_y
validgroup = test_x.groupby(['Coupon_id'])
aucs = []
for i in validgroup:
    tmpdf = i[1]
    if len(tmpdf['within_15'].unique()) < 2 or i[0] == -1.0:
        continue
    fpr, tpr, thresholds = roc_curve(tmpdf['within_15'], tmpdf['pred_prob'], pos_label=1)
    aucs.append(auc(fpr, tpr))
print(np.average(aucs))

0.5544166616848517


In [57]:
# test_df['Discount_rate'] = test_df['Discount_rate'].fillna(0)


,Coupon_id,Discount_rate,Distance,Date_received,proba,Dicount_type,Dicount_shop,Dicount_number,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
738687,-1.0,-1,0.0,0.0,0,0,-1,-1,0.0,0,0,0,0,0,0,0,0
922874,-1.0,-1,0.0,0.0,0,0,-1,-1,0.0,0,0,0,0,0,0,0,0
756426,-1.0,-1,0.0,0.0,0,0,-1,-1,0.0,0,0,0,0,0,0,0,0
345618,-1.0,-1,0.0,0.0,0,0,-1,-1,0.0,0,0,0,0,0,0,0,0
1083567,-1.0,-1,0.0,0.0,0,0,-1,-1,0.0,0,0,0,0,0,0,0,0
446969,-1.0,-1,0.0,0.0,0,0,-1,-1,0.0,0,0,0,0,0,0,0,0
892945,-1.0,-1,0.0,0.0,0,0,-1,-1,0.0,0,0,0,0,0,0,0,0
1452820,-1.0,-1,0.0,0.0,0,0,-1,-1,0.0,0,0,0,0,0,0,0,0
1351482,-1.0,-1,2.0,0.0,0,0,-1,-1,0.0,0,0,0,0,0,0,0,0
676100,-1.0,-1,0.0,0.0,0,0,-1,-1,0.0,0,0,0,0,0,0,0,0


In [71]:
test_df.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Dicount_type,Dicount_shop,Dicount_number,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,proba
0,4129537,450,9983,0.833333,1.0,20160712,1,30,5,2,0,0,1,0,0,0,0,0,0
1,6949378,1300,3429,0.833333,11.0,20160706,1,30,5,3,0,0,0,1,0,0,0,0,0


In [73]:
bb =clf.predict_proba(test_df.drop(['User_id', 'Merchant_id', 'Coupon_id', 'Date_received'], axis=1))

In [74]:
result = bb.T[0]

In [75]:
result

array([0.2756355 , 0.27755898, 0.2756355 , ..., 0.27755898, 0.12301213,
       0.2756355 ])

In [77]:
test_df = pd.read_csv('ccf_offline_stage1_test_revised.csv')
submission = pd.DataFrame({
        "User_id": test_df["User_id"],
        "Coupon_id": test_df['Coupon_id'],
        "Date_received": test_df['Date_received'],
        "result":result
    })
print(submission)
submission.to_csv('discount_ticket_3.csv', index = False)

        Coupon_id  Date_received  User_id    result
0            9983       20160712  4129537  0.275635
1            3429       20160706  6949378  0.277559
2            6928       20160727  2166529  0.275635
3            1808       20160727  2166529  0.275635
4            6500       20160708  6172162  0.275635
5            9983       20160706  4005121  0.291512
6            9983       20160716  4347394  0.123012
7           13602       20160727  3094273  0.275635
8            9983       20160729  5139970  0.275635
9           13602       20160703  3237121  0.114589
10           9983       20160716  6224386  0.114589
11          13602       20160712  6488578  0.291512
12           9983       20160703  4164865  0.114589
13           8059       20160706  4164865  0.275635
14           9983       20160713  5468674  0.277559
15           9144       20160706  6258178  0.291512
16           7341       20160727  3659521  0.260107
17          13181       20160717  3659521  0.114589
18          